In [ ]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import orth
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar
import re

class Functions:
    
    def __init__(self, f, df, d2f):
            self.f = f
            self.df = df
            self.d2f = d2f

    def zoomInt(self, phi, dphi, alpha_l, alpha_h, c1, c2, trail_step = 'bisection'):
        ''' The purpose of this algorithm is to find a suitable step length (alpha) 
        that satisfies the Wolfe conditions during a line search in an optimization problem.
        Successively decrease the inverval size of the alphas until acceptable step length found'''

        tol = np.finfo(float).eps         
        # Structure containing information about the iteration
        info = {'alpha_ls': [], 'alpha_hs': [], 'alpha_js': [], 'phi_js': [], 'dphi_js': []}

        n = 1
        stop = False
        max_iter = 100
        alpha = 0

        while n < max_iter and not stop:
            # Find trial step length alpha_j in [alpha_l, alpha_h]
            if trail_step.lower() == 'bisection':
                alpha_j = 0.5 * (alpha_h + alpha_l)
            elif trail_step.lower() == 'interp2':
                # You need to implement the 'interp2' method
                raise NotImplementedError("Interp2 method not implemented")

            phi_j = phi(alpha_j)

            # Update info
            info['alpha_ls'].append(alpha_l)
            info['alpha_hs'].append(alpha_h)
            info['alpha_js'].append(alpha_j)
            info['phi_js'].append(phi_j)

            if abs(alpha_h - alpha_l) < tol:
                alpha = alpha_j
                stop = True
                print("Line search stopped because the interval became too small. Returning center of the interval.")
                print(f'Centre: {alpha_h - alpha_l}')
                break

            if phi_j > phi(0) + c1 * alpha_j * dphi(0) or phi(alpha_j) >= phi(alpha_l):
                # alpha_j does not satisfy sufficient decrease condition look for alpha < alpha_j or phi(alpha_j) >= phi(alpha_l)
                alpha_h = alpha_j
                info['dphi_js'].append(np.nan)

            else:
                # alpha_j satisfies sufficient decrease condition
                dphi_j = dphi(alpha_j)
                info['dphi_js'].append(dphi_j)

                if abs(dphi_j) <= -c2 * dphi(0):
                    # alpha_j satisfies strong curvature condition
                    alpha = alpha_j
                    stop = True

                elif dphi_j * (alpha_h - alpha_l) >= 0:
                    # alpha_h : dphi(alpha_l)*(alpha_h - alpha_l) < 0
                    # alpha_j violates this condition but swapping alpha_l <-> alpha_h will reestablish it
                    # [alpha_j, alpha_l]
                    alpha_h = alpha_l
                
                alpha_l = alpha_j

            n += 1

        return alpha, info
    
    def lineSearch(self, x_k, p_k, alpha_max, alpha0, opts={'c1': 1e-4, 'c2': 0.9}):

        # Parameters
        w = 0.9

        ## Function phi(alpha) = f(x_k + alpha*p_k)
        def phi(alpha):
            return self.f(x_k + alpha * p_k)

        ## Derivative dphi(alpha) = df(x_k + alpha*p_k)' * p_k
        def dphi(alpha):
            return self.df(x_k + alpha * p_k).T @ p_k

        # Initialization
        alpha = [alpha0, alpha_max] # alpha0 should be 0
        phi_i = [phi(0)]
        dphi_i = [dphi(0)]
        alpha_s = 0
        n = 1
        max_iter = 10
        stop = False

        while n < max_iter and not stop:
            phi_i.append(phi(alpha[n]))
            dphi_i.append(dphi(alpha[n]))
            # check if alpha(n) satisfies the Wolfe conditions
            if (phi_i[n] > phi_i[0] + opts['c1'] * alpha[n] * dphi_i[0] or (phi_i[n] >= phi_i[n - 1] and n > 1)):
                alpha_s, _ = Functions.zoomInt(self, phi, dphi, alpha_l = alpha[n - 1], alpha_h = alpha[n], c1=opts['c1'], c2=opts['c2'], trail_step = 'bisection')
                stop = True
            #evaluate phi'(alpha_i) - check strong curvature condition:
            elif (abs(dphi_i[n]) <= opts['c2'] * dphi_i[0]):
                alpha_s = alpha[n]
                stop = True
            elif (dphi_i[n] >= 0):
                alpha_s, _ =  Functions.zoomInt(self, phi, dphi, alpha_l=alpha[n], alpha_h=alpha[n-1], c1=opts['c1'], c2=opts['c2'])
                stop = True

            # Choose alpha(n+1) in (alpha(n), alpha_max)
            alpha.append(w * alpha[n] + (1 - w) * alpha_max)
            n += 1

            # Maximal number of iterations reached. Return the last alpha(n+1).
            if n == max_iter:
                alpha_s = alpha[n]
        
        return alpha_s, {'alphas': alpha}
    
            
    def backtracking(self, x_k, p, alpha0, opts={'c1': 1e-4, 'rho': None}):

        if opts['c1'] == None:
            c1 = 1e-4
        else:
            c1 = opts['c1']
        # rho = 0.1 for steepest descent, conjugate gradients
        # rho = 0.9 for Newton, Quasi-Newton
        if opts['rho'] == None:
            rho = 0.2
        else:
            rho = opts['rho']
        if alpha0 == None:
            alpha0 = 1


        # Initialize info structure
        info = {'alphas': [alpha0], 'rho': [rho], 'c1': [c1]}

        # Initial step length
        alpha = alpha0

        # Compute f, grad f at x_k
        f_k = self.f(x_k)
        df_k = self.df(x_k)

        # Backtracking line search
        while self.f(x_k + alpha * p) > f_k + c1 * alpha * np.dot(df_k, p):
            alpha = rho * alpha
            info['alphas'].append(alpha)

        return alpha, info
    

    def back_stepsize(self, x_k, p_k, alpha0, c, p):
        # Function phi(alpha) = f(x_k + alpha*p_k)
        def phi(alpha):
            return self.f(x_k + alpha * p_k)

        # Derivative dphi(alpha) = df(x_k + alpha*p_k)' * p_k
        def dphi(alpha):
            return np.dot(self.df(x_k + alpha * p_k).T, p_k) # wrap in list??

        alpha = [alpha0]
        phi_i = [phi(0)]
        dphi_i = [dphi(0)]
        alpha_s = 0
        n = 0
        max_iter = 10
        stop = False

        while n < max_iter and not stop:

            # check if alpha(n) satisfies the sufficient decrease condition
            if phi(alpha[n]) <= (phi(0) + c * alpha[n] * dphi(alpha[n])):
                alpha_s = alpha[n] * p
                stop = True

            # Choose alpha(n+1) in (alpha(n), alpha_max)
            alpha.append(alpha[n])
            phi_i.append(phi(alpha[n]))
            dphi_i.append(dphi(alpha[n]))

            n += 1

            # Maximal number of iterations reached. Return the last alpha(n+1).
            if n == max_iter:
                print('Max iterations reached, setting alpha = alpha[n]*p')
                alpha_s = alpha[n] * p
        
        return alpha_s, {'alphas': alpha}
    

    def descentLineSearch(self, x0, alpha0, alpha_max, tol, maxIter, ls ='linesearch', switch='steepest',
                        stop_type = 'grad', fixed_step = False, fixed_alpha=0.1):

        # Initialization
        nIter = 0
        x_k = x0
        info = {'xs': [np.array(x0)], 'alphas': [alpha0]}
        stopCond = False

        if not fixed_step:
            # Loop until convergence or maximum number of iterations
            while not stopCond and nIter <= maxIter:

                nIter += 1

                # Compute descent direction
                if switch.lower() == 'steepest':
                    # p_k steepest descent direction
                    p_k = -self.df(x_k)
                elif switch.lower() == 'newton':                    
                    p_k = -np.linalg.inv(self.d2f(x_k)).dot(self.df(x_k)) # solution to Ax = b is x = A\b
                else:
                    raise ValueError(f"Unknown descent method '{switch}'. Expected 'steepest' or 'newton'.")

                # Call line search to compute step length alpha_k
                if ls.lower() =='linesearch':
                    alpha_k, _ = Functions.lineSearch(x_k=x_k, alpha0=alpha0, p_k=p_k, 
                                alpha_max=alpha_max, opts = {'c1': 1e-4, 'c2': 0.9}) 
                    
                elif ls.lower() == 'backtracking':
                    alpha_k, _ = Functions.backtracking(x_k, p_k, alpha0, opts={'c1': 1e-4, 'rho': 0.2})
                    
                
                # Update x_k
                x_k_1 = x_k
                x_k = x_k + alpha_k * p_k

                # Store iteration info
                info['xs'].append(x_k)
                info['alphas'].append(alpha_k)

                # Stopping conditions
                if stop_type.lower() == 'step':
                    # Compute relative step length
                    normStep = norm(x_k - x_k_1) / norm(x_k_1)
                    stopCond = (normStep < tol)
                elif stop_type.lower() == 'grad':
                    stopCond = (norm(self.df(x_k)) < tol * (1 + abs(self.f(x_k)))) # np.inf


            # Assign output values
            xMin = x_k
            fMin = self.f(x_k)

        else:
            while not stopCond and nIter <= maxIter:

                nIter += 1
                # Compute descent direction
                if switch.lower() == 'steepest':
                    # p_k steepest descent direction
                    p_k = -self.df(x_k)
                elif switch.lower() == 'newton':
                    # p_k Newton direction
                    p_k = -np.linalg.inv(self.d2f(x_k)).dot(self.df(x_k))

                alpha_k = fixed_alpha

                # Update x_k
                x_k_1 = x_k
                x_k = x_k + alpha_k * p_k

                # Store iteration info
                info['xs'].append(x_k)
                info['alphas'].append(alpha_k)

                # Stopping conditions
                if stop_type.lower() == 'step':
                    # Compute relative step length
                    normStep = norm(x_k - x_k_1) / norm(x_k_1)
                    stopCond = (normStep < tol)
                elif stop_type.lower() == 'grad':
                    stopCond = (norm(self.df(x_k)) < tol * (1 + abs(self.f(x_k)))) # np.inf


            # Assign output values
            xMin = x_k
            fMin = self.f(x_k)

        return xMin, fMin, nIter, info
    

    def backtrack_LineSearch(self, x0, alpha0=1, tol=0.1, maxIter=1000, switch='steepest', 
                            stop_type = 'grad', c1=1, c2=0.1, p=0.2):

        # Initialization
        nIter = 0
        x_k = x0
        info = {'xs': [np.array([x0])], 'alphas': [alpha0]}
        stopCond = False
        opts = {}

        # Loop until convergence or maximum number of iterations
        while not stopCond and nIter <= maxIter:

            nIter += 1

            # Compute descent direction
            if switch.lower() == 'steepest':
                # p_k steepest descent direction
                p_k = -1 * self.df(x_k)
                alpha0 = 1
            elif switch.lower() == 'newton':
                # p_k = -1 * d2f(x_k)^-1 / df(x_k)
                p_k = -np.linalg.inv(self.d2f(x_k)).dot(self.df(x_k))
                # c2 = 1-c1
                # solution to Ax = b is x = A\b

            # Call line search to compute step length alpha_k
            alpha_k, _ = Functions.back_stepsize(x_k, p_k, alpha0, c=c1, p=p) # or backtracking(self, x_k, p, alpha0, opts={'c1': 1e-4, 'rho': 0.9})
            # back_stepsize(self, x_k, p_k, alpha0, c, p)

            # Update x_k
            x_k_1 = x_k
            x_k = x_k + alpha_k * p_k

            # Store iteration info
            info['xs'].append(x_k)
            info['alphas'].append(alpha_k)

            # Stopping conditions
            if stop_type == 'step':
                # Compute relative step length
                normStep = norm(x_k - x_k_1) / norm(x_k_1)
                stopCond = (normStep < tol)
            elif stop_type == 'grad':
                stopCond = (norm(self.df(x_k)) < tol * (1 + abs(self.f(x_k)))) # np.inf


        # Assign output values
        xMin = x_k
        fMin = self.f(x_k)

        return xMin, fMin, nIter, info
    

    def root_finder(t, p_U, a, trust_radius_Delta):
        eqn = np.linalg.norm(p_U + (t-1)*(a - p_U))**2 - trust_radius_Delta**2

        # Find the root using scipy.optimize.root_scalar
        result = root_scalar(eqn, bracket=[0, 5], method='bisect', x0=1.5)
        tau2 = result.root 
        return tau2
   
   
    def trustRegion(self, x0, p, trust_radius_Delta, eta, 
                    tol, maxIter, stopType = 'grad'): 
        
        # Initialisation
        current_radius = 0.5 * trust_radius_Delta
        stopCond = False
        x_k = x0
        nTaken = 0
        k = 0

        info = {'xs':[np.array(x_k)], 'xind':[], 'rhos':[], 'Deltas':[], 'stopCond':[]}
        info['xind'] = np.zeros(maxIter+1)

        while not stopCond and (k < maxIter):
            k += 1
        
            # Construct and solve quadratic model  
            def m_k(x_k, p): #initial_radius=initial_radius
                # if norm(p) <= initial_radius:
                return self.f(x_k) + self.df(x_k).T @ p + 0.5 * p.T @ self.d2f(x_k) @ p

            # Evaluate actual to predicted reduction ratio
            # Bk = d2f(x_k) for trust-region Newton method
            acc_reduction = self.f(x_k) - self.f(x_k + p)
            pred_reduction = m_k(x_k, np.zeros(np.shape(p))) - m_k(x_k, p=np.array(p))
            p_k = acc_reduction/pred_reduction 
            # Note that since the step pk is obtained by minimizing the model mk over a region 
            # that includes the step p = 0, the predicted reduction will always be nonnegative.
            # Thus if ρk is negative, the new objective value f (xk + pk) is greater than the 
            # current value f (xk), so the step must be rejected.
            # if ρk is close to 1, there is good agreement between the model mk and the function 
            # over this step, so it is safe to expand the trust region for the next iteration.


            # Record iteration information
            info['rhos'].append(p_k)
            info['Deltas'].append(current_radius)

            # Adjust the size of the trustregion
            if p_k < 0.25:
                current_radius = 0.25 * current_radius
            elif p_k > 0.75 and norm(p) == current_radius: # or abs(p.T @ p - Delta_k**2) < 1e-16: # norm(p) == current_radius is a safeguard against increasing Delta_k indefinitely, norm(p)=abs(p.T * p - Delta_k**2)
                current_radius = min(2*current_radius, trust_radius_Delta)
            else:
                current_radius = current_radius 
            
            # Accept step
            if p_k > eta:
                x_k_1 = x_k + p_k
            else:
                x_k_1 = x_k
                # nTaken = nTaken + 1;
                # info.xs(:,nTaken+1) = x_k;
                # info.xind(nTaken+1) = k;


            # Evaluate stopping condition: 
            if stopType == 'step':
                # relative step length
                stopCond = (norm(x_k - x_k_1)/norm(x_k_1) < tol)
            elif stopType == 'grad':
                # gradient norm
                #stopCond = (norm(F.df(x_k)) < tol);
                stopCond = (norm(self.df(x_k), ord=np.inf) < tol*(1 + abs(self.f(x_k))))
            elif current_radius < 1e-6*trust_radius_Delta:
                # Stop iteration if Delta_k shrank below 1e-6*Delta. Otherwise, if the model 
                # does not improve inspite of shinking, the algorithm would shrink Delta_k indefinitely.
                print('Region of interest is to small. Terminating iteration.')

            f_k = self.f(x_k)
            info['stopCond'].append(stopCond)
            info['xs'].append(x_k)
            info['xind'][k] = nTaken
            # info['xs'][:,nTaken+2:] = []
            # info['xind'][nTaken+2:] = []
            # info['rhos'][k+1:] = []  
            # info['Deltas'][k+1:] = [] 

        return x_k, f_k, k, info 


    def flecther_reeves(self, x0, alpha0, alpha_max, tol, maxIter,
                        ls ='linesearch', stop_type = 'grad', opts = {'c1': 1e-4, 'c2': 0.1}):

        # Initialization
        nIter = 0
        x_k = x0
        p_k = -self.df(x_k)
        info = {'xs': [np.array(x_k)], 'alphas': [alpha0], 'betas': [], 'grads': [self.df(x_k)]}
        stopCond = False

        # Loop until convergence or maximum number of iterations
        while not stopCond and nIter <= maxIter:

            # Call line search to compute step length alpha_k
            if ls.lower() =='linesearch':
                alpha_k, _ = Functions.lineSearch(self, x_k=x_k, p_k=p_k, alpha0=alpha0,
                            alpha_max=alpha_max, opts = opts) 
                
            elif ls.lower() == 'backtracking':
                alpha_k, _ = Functions.backtracking(self, x_k, p_k, alpha0, opts={'c1': 1e-4, 'rho': 0.9})
                
            # Update x
            x_k_1 = x_k + alpha_k * p_k

            # Update gradient  
            grad = self.df(x_k_1)
            info['grads'].append(grad)

            # FR Method
            beta = np.max((info['grads'][nIter+1].T * info['grads'][nIter+1]) / (info['grads'][nIter].T * info['grads'][nIter]), 0)

            # # Compute new direction p1
            p_k = -grad + beta * p_k

            info['betas'].append(beta)
            info['xs'].append(x_k_1)
            info['alphas'].append(alpha_k)

            # Stopping conditions
            if stop_type.lower() == 'step':
                # Compute relative step length
                normStep = norm(x_k - x_k_1) / norm(x_k_1)
                stopCond = (normStep < tol)
            if stop_type.lower() == 'grad':
                stopCond = (norm(self.df(x_k), np.inf)) < tol * (1 + abs(self.f(x_k)))

            x_k = x_k_1
            nIter += 1

        # Assign output values
        xMin = x_k
        fMin = self.f(x_k)

        return xMin, fMin, nIter, info
    

    def polak_ribiereCGmethod(self, x0, alpha0, tol, maxIter, alpha_max,
                        ls ='linesearch', stop_type = 'grad'):

        # Initialization
        nIter = 0
        x_k = x0
        p_k = -self.df(x_k)
        info = {'xs': [np.array(x_k)], 'alphas': [alpha0], 'betas': [], 'grads': [self.df(x_k)]}
        stopCond = False

        # Loop until convergence or maximum number of iterations
        while not stopCond and nIter <= maxIter:

            # Call line search to compute step length alpha_k
            # print(x_k)
            if ls.lower() =='linesearch':
                alpha_k, _ = Functions.lineSearch(self, x_k=x_k, p_k=p_k, alpha0=alpha0,
                            alpha_max=alpha_max, opts = {'c1': 1e-4, 'c2': 0.1}) 
                
            elif ls.lower() == 'backtracking':
                alpha_k, _ = Functions.backtracking(self, x_k, p_k, alpha0, opts={'c1': 1e-4, 'rho': 0.2})
                
            # Update x
            x_k_1 = x_k + alpha_k * p_k

            # Update gradient  
            grad = self.df(x_k_1)
            info['grads'].append(grad)

            # Polak Method
            beta = (info['grads'][nIter+1].T * (info['grads'][nIter+1] - info['grads'][nIter])) / norm(info['grads'][nIter])**2

            # # Compute new direction p1
            p_k = -grad + beta * p_k

            info['betas'].append(beta)
            info['xs'].append(x_k_1)
            info['alphas'].append(alpha_k)

            # Stopping conditions
            if stop_type.lower() == 'step':
                # Compute relative step length
                normStep = norm(x_k - x_k_1) / norm(x_k_1)
                stopCond = (normStep < tol)
            if stop_type.lower() == 'grad':
                stopCond = (norm(self.df(x_k), np.inf)) < tol * (1 + abs(self.f(x_k)))

            x_k = x_k_1
            nIter += 1

        # Assign output values
        xMin = x_k
        fMin = self.f(x_k)

        return xMin, fMin, nIter, info
    

    def BFGS_alg(self, x0, alpha0, tol, maxIter, stop_type = 'grad', 
                 ls='linesearch', alpha_max=None):

        # Initialization
        nIter = 0
        x_k = x0
        info = {'xs': [x0], 'alphas': [alpha0]}
        stopCond = False
        H0 = self.d2f(x_k) # np.eye(2)
        rho_k = 0.9
            
        # Loop until convergence or maximum number of iterations
        while not stopCond and nIter <= maxIter and norm(self.df(x_k)) > tol:

            p_k = -H0.dot(self.df(x_k))
            
            # Call line search to compute step length alpha_k
            if ls.lower() =='linesearch':
                alpha_k, _ = Functions.lineSearch(self, x_k=x_k, p_k=p_k, alpha0=alpha0, 
                            alpha_max=alpha_max, opts = {'c1': 1e-4, 'c2': 0.99}) 
            else:
                print('backtracking takes too long')

            # Update x_k
            x_k_1 = x_k + alpha_k * p_k

            # Compute step diff s_k
            s_k = x_k - x_k_1
            
            # must satify curvature condition s_k.T * y_k > 0
            
            # Compute the gradient difference g_k
            y_k = self.df(x_k) - self.df(x_k_1)

            # Compute rho
            rho_k = 1/(y_k.T @ s_k)

            # Compute the new inverse hessian
            H1 = (np.eye(2) - rho_k * s_k @ y_k.T) @ H0 @ (np.eye(2) - rho_k * y_k @ s_k.T) + rho_k * (s_k @ s_k.T)
            
            H0 = H1
            x_k = x_k_1
            nIter += 1
            
            # Store iteration info
            info['xs'].append(x_k)
            info['alphas'].append(alpha_k)

            # Stopping conditions
            if stop_type.lower() == 'step':
                # Compute relative step length
                normStep = norm(x_k - x_k_1) / norm(x_k_1)
                stopCond = (normStep < tol)
            elif stop_type.lower() == 'grad':
                stopCond = (norm(self.df(x_k),  np.inf) < tol * (1 + abs(self.f(x_k)))) # np.inf


        # Assign output values
        xMin = x_k
        fMin = self.f(x_k)

        return xMin, fMin, nIter, info


    def convergence_history(self, info, xMin, x0, p, H=None, plot_type=None):
        
        # convert info['xs'] to numpy array
        arr = np.zeros(shape=(2,len(info['xs'])))
        for i in range(len(info['xs'])):
            x = info['xs'][i][0]
            y = info['xs'][i][1]
            arr[0][i] = x
            arr[1][i] = y
        
        if xMin is None:
            xMin = (arr[0][-1], arr[1][-1])
        
        shape = arr.shape[1]
        if p == 'M':
            p = 2
            if H is not None:
                M = H
            else:
                M = self.d2f(xMin)  # M is the Hessian at the solution, M has to be symmetric positive definite
            
            # Convergence of iterates: || x_k - xMin ||_M
            err = info['xs'] - np.tile(xMin, (2, shape))            
            # err = xs - np.tile(x_min[:, np.newaxis], (1, xs.shape[1]))
            con_x = [np.sqrt(np.dot(err[k].T, M.dot(err[k]))) for k in range(shape)]
        else:
            # Convergence of iterates: || x_k - xMin ||_p
            # print((arr - np.array([xMin[0], xMin[1]]*shape).reshape(2,shape)).shape)
            con_x = np.sum(np.abs(arr - np.array([xMin[0], xMin[1]]*shape).reshape(2,shape))**p, axis=0)**(1/p)

        if self.f is not None:
            # Convergence of function values: f(x_k) - f(xMin)
            con_f = [self.f([arr[0,k], arr[1,k]]) - self.f(xMin) for k in range(shape)]

            # Convergence of gradient: || f(x_k)||_p
            con_df = [np.sum(np.abs(self.df([arr[0,k], arr[1,k]]))**p)**(1/p) for k in range(shape)]
        else:
            con_f = []
            con_df = []


        conInfo = {'x': con_x, 'f': con_f, 'df': con_df} # con

        if plot_type == 'Q-convergence':

            # Q convergence 
            p = 1
            rs1 = conInfo['x'][1:] / (conInfo['x'][:-1]**p)
            ps = 1.4
            rss = conInfo['x'][1:] / (conInfo['x'][:-1]**ps)
            p = 2
            rs2 = conInfo['x'][1:] / (conInfo['x'][:-1]**p)
            
            plt.figure()
            plt.plot(rs1)
            plt.title('Q convergence')
            plt.legend(['p=1'])
            plt.grid(True)

            plt.figure()
            plt.plot(rs1)
            plt.plot(rss, 'k--')
            plt.title('Q convergence')
            plt.legend(['p=1', 'p='+str(ps)])
            plt.grid(True)

            plt.figure()
            plt.plot(rs1)
            plt.plot(rss, 'k--')
            plt.plot(rs2, 'r-.')
            plt.xlabel('k')
            plt.ylabel(r'$\frac{||x_{k+1} - x^* ||}{||x_k - x^*||^p}$')
            plt.title('Q convergence')
            plt.legend(['p=1', 'p='+str(ps), 'p=2'])
            plt.grid(True)

        elif plot_type == 'Algebraic_convergence':
            # Algebraic convergence
            plt.figure()
            plt.loglog(conInfo['x'])
            plt.title('Algebraic convergence')
            plt.xlabel('ln(k)')
            plt.ylabel('ln(||x_k - x*||)')
            plt.grid(True)  
        
        elif plot_type == 'Exp_convergence':
            # Exponential convergence
            plt.figure()
            plt.semilogy(conInfo['x'])
            plt.title('Exponential convergence')
            plt.xlabel('k')
            plt.ylabel('ln(||x_k - x*||)')
            plt.xlim(0, len(conInfo['x'])-1)
            plt.grid(True)

        elif plot_type == 'Grad_over_iters':
            # Gradient norm over iterates
            plt.figure()
            plt.semilogy(conInfo['df'])
            plt.title('Exponential convergence: ||grad(f)||')
            plt.xlabel('k')
            plt.ylabel('ln||grad(x_k)||')
            plt.grid(True)

        elif plot_type == 'Function_values':
            # Gradient norm convergence to a stationary point
            sdfConSteep = [sorted(conInfo['df'], reverse=True)]
            min_dfj = np.zeros(len(sdfConSteep))

            plt.figure()
            for j in range(len(sdfConSteep)):
                min_dfj[j] = sdfConSteep[j][j]
            plt.loglog(range(1, len(sdfConSteep)+1), min_dfj)


            L = np.max(np.abs(np.linalg.eig(self.d2f(x0)))) # Lipschitz constant 
            plt.loglog(range(2, len(sdfConSteep)+1, 2), np.sqrt(2 * 2 * (self.f(x0) - self.f(info['xs'][-1])) / range(2, len(sdfConSteep)+1, 2)), 'rx-')
            plt.loglog(range(2, len(sdfConSteep)+1, 2), 4 * 2 * np.linalg.norm(x0 - info['xs'][-1]) / range(2, len(sdfConSteep)+1, 2), 'ko-')
            plt.grid(True)
            plt.title(r"Approximate convergence $\min_{1,...,k} \|\nabla (f (x_k) )\|$",
                    fontdict={'verticalalignment': 'bottom'}, fontsize=12, position=(0.5, 0.93))
            plt.xlabel('ln k')
            plt.ylabel(r'ln $\min_{1,...,k} \| \nabla(f(x_{k})) \|$', fontsize=12)
        
        elif plot_type == None:
            return {'x': con_x, 'f': con_f, 'df': con_df}

    
    def approximate_grad_convergence(self, frame1, coord1, coord2=None, num_functions=2, frame2=None, title_plot='Dog leg method'):
        if num_functions == 2:
            ys_1 = []
            for i in range(len(frame1)):
                ys_1.append(norm(self.df(frame1[i])))
            x_1 = np.arange(1,len(frame1)+1)

            ys_2 = []
            for i in range(len(frame2)):
                ys_2.append(norm(self.df(frame2[i])))
            x_2 = np.arange(1,len(frame2)+1)

            plt.figure()
            plt.loglog(x_1, ys_1, label=f'Coordinate {coord1}')
            plt.loglog(x_2, ys_2, label=f'Coordinate {coord2}')
            plt.title(f'Gradient Norm Convergence rate of {title_plot}')
            plt.xlabel(r'$\ln k$')
            plt.ylabel(r'$\ln||\nabla f(x,y)$||')
            plt.xlim(1,max(x_1[-1],x_2[-1]))
            plt.legend()

        elif num_functions == 1:
            ys = []
            for i in range(len(frame1)):
                ys.append(norm(self.df(frame1[i])))
            x = np.arange(1,len(frame1)+1)

            plt.figure()
            plt.loglog(x, ys, label=f'Coordinate {coord1}')
            plt.title('Gradient Norm Convergence rate of Newton Method')
            plt.xlabel('Iterations')
            plt.ylabel(r'||$\nabla f(x,y)$||')
            plt.xlim(1,x[-1])
            plt.legend()


    def file_formatting(self,file):

        string_values = file[:, 1][0].strip('][')
        numeric_values = re.findall(r'[-+]?\d*\.\d+|\d+', string_values)
        xs = np.array([float(value) for value in numeric_values])
        xs = xs.reshape(-1, 2)

        deltas = file[:, 0][0].strip('][')
        deltas = deltas.split(',')
        deltas = [float(i) for i in deltas]
        deltas = np.array(deltas)

        info = {'xs': [], 'deltas': [], 'xind': []}
        for i in range(len(xs)):
            info['xs'].append(xs[i])
            info['deltas'].append(deltas[i])
            info['xind'].append(i)

        return info


    def visualize_convergence(self, info, X, Y, Z, mode, max_steps=None):
        TIME_PAUSE = 0.5
        if max_steps is None:
            MAX_STEPS = len(info['xs'])
        else:
            MAX_STEPS = min(max_steps, len(info['xs']))

        plt.figure()
        plt.contourf(X, Y, Z, 20)
        bbox = plt.axis()

        if mode == 'final':
            plt.plot(info['xs'][:][0], info['xs'][:][1], '-or', linewidth=2, markersize=3)
            plt.title('Convergence')
        elif mode == 'iterative':
            n_iter = np.shape(info['xs'])[1]

            for j in range(MAX_STEPS):
                plt.clf()
                plt.contour(X, Y, Z, 20)
                plt.plot(info['xs'][j + 1][0], info['xs'][j + 1][1], '-or', linewidth=2, markersize=3)
                plt.plot(info['xs'][j][0], info['xs'][j][1], '-xr', linewidth=2, markersize=8)

                if 'Deltas' in info and j > 0:
                    plt.plot(info['xs'][j - 1][0] + np.cos(np.arange(0, 2 * np.pi, 0.05)) * info['Deltas'][info['xind'][j]],
                            info['xs'][j - 1][1] + np.sin(np.arange(0, 2 * np.pi, 0.05)) * info['Deltas'][info['xind'][j]],
                            ':k', linewidth=2)

                plt.axis(bbox)
                plt.title(f'Convergence: steps 1 : {j + 1}')
                plt.pause(TIME_PAUSE)